[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/agents/agent_fireworks_ai_langchain_mongodb.ipynb)

## Install Libraries

In [ ]:
!pip install langchain langchain_openai langchain-fireworks langchain-mongodb tavily-python arxiv pymupdf datasets pymongo

## Set Evironment Variables

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["FIREWORKS_API_KEY"] = ""
os.environ["MONGO_URI"] = ""

FIREWORKS_API_KEY = os.environ.get("FIREWORKS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MONGO_URI = os.environ.get("MONGO_URI")

## Data Ingestion into MongoDB Vector Database


In [ ]:
import pandas as pd
from datasets import load_dataset

data = load_dataset("MongoDB/subset_arxiv_papers_with_emebeddings")
dataset_df = pd.DataFrame(data["train"])

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(len(dataset_df))
dataset_df.head()

50000


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,embedding
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...","[0.0594153292, -0.0440569334, -0.0487333685, -..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]","[0.0247399714, -0.065658465, 0.0201423876, -0...."
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]","[0.0491479263, 0.0728017688, 0.0604138002, 0.0..."
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]","[0.0389556214, -0.0410280302, 0.0410280302, -0..."
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]","[0.118412666, -0.0127423415, 0.1185125113, 0.0..."


In [ ]:
from pymongo import MongoClient

# Initialize MongoDB python client
client = MongoClient(MONGO_URI)

DB_NAME = "agent_demo"
COLLECTION_NAME = "knowledge"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
collection = client[DB_NAME][COLLECTION_NAME]

In [ ]:
# Delete any existing records in the collection
collection.delete_many({})

# Data Ingestion
records = dataset_df.to_dict('records')
collection.insert_many(records)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## Create Vector Search Index Defintion

```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 256,
      "similarity": "cosine"
    }
  ]
}
```

## Create LangChain Retriever (MongoDB)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=256)

# Vector Store Creation
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGO_URI,
    namespace=DB_NAME + "." + COLLECTION_NAME,
    embedding= embedding_model,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key="abstract"
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

## Configure LLM Using Fireworks AI

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_fireworks import Fireworks, ChatFireworks

llm = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v1",
    max_tokens=256)

## Agent Tools Creation

In [ ]:
from langchain.agents import tool, Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.document_loaders import ArxivLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Custom Tool Definiton
@tool
def get_metadata_information_from_arxiv(word: str) -> list:
  """
  Fetches and returns metadata for a maximum of ten documents from arXiv matching the given query word.

  Args:
    word (str): The search query to find relevant documents on arXiv.

  Returns:
    list: Metadata about the documents matching the query.
  """
  docs = ArxivLoader(query=word, load_max_docs=10).load()
  # Extract just the metadata from each document
  metadata_list = [doc.metadata for doc in docs]
  return metadata_list


@tool
def get_information_from_arxiv(word: str) -> list:
  """
  Fetches and returns metadata for a single research paper from arXiv matching the given query word, which is the ID of the paper, for example: 704.0001.

  Args:
    word (str): The search query to find the relevant paper on arXiv using the ID.

  Returns:
    list: Data about the paper matching the query.
  """
  doc = ArxivLoader(query=word, load_max_docs=1).load()
  return doc


retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="knowledge_base",
    description="This serves as the base knowledge source of the agent and contains some records of research papers from Arxiv. This tool is used as the first step for exploration and reseach efforts."
)

In [ ]:
tools = [retriever_tool, get_metadata_information_from_arxiv, get_information_from_arxiv]

## Agent Prompt Creation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_purpose = "You are a helpful research assistant"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", agent_purpose),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad")
    ]
)

## Agent Memory Using MongoDB

In [ ]:
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
        return MongoDBChatMessageHistory(MONGO_URI, session_id, database_name=DB_NAME, collection_name="history")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=get_session_history("my-session")
)

## Agent Creation

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

## Agent Exectution

In [ ]:
agent_executor.invoke({"input": "Get me a list of research papers on the topic Prompt Compression"})



> Entering new AgentExecutor chain...

Invoking: `knowledge_base` with `{'query': 'Prompt Compression'}`


  Computation on compressed strings is one of the key approaches to processing
massive data sets. We consider local subsequence recognition problems on
strings compressed by straight-line programs (SLP), which is closely related to
Lempel--Ziv compression. For an SLP-compressed text of length $\bar m$, and an
uncompressed pattern of length $n$, C{\'e}gielski et al. gave an algorithm for
local subsequence recognition running in time $O(\bar mn^2 \log n)$. We improve
the running time to $O(\bar mn^{1.5})$. Our algorithm can also be used to
compute the longest common subsequence between a compressed text and an
uncompressed pattern in time $O(\bar mn^{1.5})$; the same problem with a
compressed pattern is known to be NP-hard.


  A new incremental algorithm for data compression is presented. For a sequence
of input symbols algorithm incrementally constructs a p-adic integer number a

{'input': 'Get me a list of research papers on the topic Prompt Compression',
 'chat_history': 'Human: Get me a list of research papers on the topic Prompt Compression\nAI: In the document \'A Comprehensive Study of Prompt Compression for LLMs,\' the authors propose a novel prompt compression method called prompt compression via relation-aware graph (PROMPT-SAW). The PROMPT-SAW algorithm uses a graph-based approach to compress large prompts into shorter ones while preserving contextual coherence and reducing redundancy. It first extracts all entities and their relations from the given prompt to construct a graph, and then uses the graph to find small-scale information units that contain less but still meaningful information.\nThe authors also evaluate the performance of PROMPT-SAW through experiments on three different tasks: covert character sequence decoding, short answer generation, and summarization. They find\nHuman: Get me the abstract of the first paper on the list\nAI: <plain>T

In [ ]:
agent_executor.invoke({"input":"Get me the abstract of the first paper you found"})



> Entering new AgentExecutor chain...

Invoking: `get_metadata_information_from_arxiv` with `{'word': 'first paper'}`


[{'Published': '2012-07-27', 'Title': 'First Stars IV: Summary Talk', 'Authors': 'Andrea Ferrara', 'Summary': 'The paper contains the summary of the First Stars IV 2012 Conference held in\nKyoto, Japan'}, {'Published': '2020-10-04', 'Title': 'Some inequalities between Laplacian eigenvalues on Riemannian manifolds', 'Authors': 'Guangyue Huang, Xuerong Qi', 'Summary': 'In this paper, we study a first Dirichlet eigenfunction of the weighted\n$p$-Laplacian on a bounded domain in a complete weighted Riemannian manifold.\nBy constructing gradient estimates for a first eigenfunction, we obtain some\nrelationships between weighted $p$-Laplacian first eigenvalues. As an immediate\napplication, we also obtain some eigenvalue comparison results between the\nfirst Dirichlet eigenvalue of the weighted Laplacian, the first clamped plate\neigenvalue and the first buckling eigenval

{'input': 'Get me the abstract of the first paper on the list',
 'chat_history': 'Human: Get me a list of research papers on the topic Prompt Compression\nAI: In the document \'A Comprehensive Study of Prompt Compression for LLMs,\' the authors propose a novel prompt compression method called prompt compression via relation-aware graph (PROMPT-SAW). The PROMPT-SAW algorithm uses a graph-based approach to compress large prompts into shorter ones while preserving contextual coherence and reducing redundancy. It first extracts all entities and their relations from the given prompt to construct a graph, and then uses the graph to find small-scale information units that contain less but still meaningful information.\nThe authors also evaluate the performance of PROMPT-SAW through experiments on three different tasks: covert character sequence decoding, short answer generation, and summarization. They find\nHuman: Get me the abstract of the first paper on the list\nAI: <plain>The abstract of